In [68]:
class Pulse():
    def __init__(self, source, destination, is_high) -> None:
        self.source = source
        self.destination = destination
        self.is_high = is_high

    def __repr__(self) -> str:
        return f"{self.source.name if self.source else 'Button'} -{'high' if self.is_high else 'low'}-> {self.destination.name}"

In [69]:
class FlipFlop():
    def __init__(self, name, destinations) -> None:
        self.is_on = False
        self.name = name
        self.destinations = destinations
        self.type ="flipflop"

    def handle_pulse(self, pulse: Pulse):
        new_pulses = []
        if not pulse.is_high:
            self.is_on = not self.is_on
            for destination in self.destinations:
                new_pulses.append(Pulse(self, destination, self.is_on))
        return new_pulses

In [70]:
class Broadcaster():
    def __init__(self, name, destinations) -> None:
        self.destinations = destinations
        self.name = name
        self.type ="broadcaster"

    def handle_pulse(self, pulse: Pulse):
        new_pulses = []
        for destination in self.destinations:
            new_pulses.append(Pulse(self, destination, pulse.is_high))
        return new_pulses

In [71]:
class Conjunction():
    def __init__(self, name, destinations) -> None:
        self.destinations = destinations
        self.name = name
        self.inputs = {}
        self.type ="conjunction"

    def register_input(self, source):
        self.inputs[source.name] = False


    def handle_pulse(self, pulse: Pulse):
        new_pulses = []
        self.inputs[pulse.source.name] = pulse.is_high
        all_true = sum((self.inputs.values())) == len(self.inputs)
        for destination in self.destinations:
            new_pulses.append(Pulse(self, destination, not all_true))
        return new_pulses

In [72]:
parsing_items = False
with open("day20.txt") as f:
    lines = f.readlines()
nodes = {}
for i, line in enumerate(lines):
    if len(line) > 3:
        if i != len(lines) -1:
            line = line[:-1]
        #print(line)
        destinations = line.split(">")[1].split(",")
        destinations = [dest.strip() for dest in destinations]
        name = line.split()[0][1:]
        if line.startswith("broadcaster"):
            nodes["broadcaster"] = Broadcaster("broadcaster", destinations)
        elif line.startswith("%"):
            nodes[name] = FlipFlop(name, destinations)
        elif line.startswith("&"):
            nodes[name] = Conjunction(name, destinations)

for node in nodes.values():
    node.destinations = [nodes[dest] if dest in nodes else Broadcaster(dest, []) for dest in node.destinations]
    for destination in node.destinations:
        if destination.type == "conjunction":
            destination.register_input(node)

In [73]:
#nodes["inv"].inputs

In [74]:
from collections import deque
queue = deque()
total_lows = 0
total_highs= 0
for i in range(1000):
    #print("##########")
    queue.append(Pulse(None, nodes["broadcaster"], False))
    while (len(queue)):
        signal = queue.popleft()
        #print(signal)
        if signal.is_high:
            total_highs+=1
        else:
            total_lows+=1
        new_signals = signal.destination.handle_pulse(signal)
        for new_signal in new_signals:
            queue.append(new_signal)
total_lows * total_highs


898731036